In [1]:
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline, BSpline, splev, splrep, make_interp_spline
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point
from shapely.ops import split, snap, linemerge

from core.lanelet_helpers import LaneletHelpers
from core.scenario import Frame, AgentState, Scenario
from lanelet_test_helpers import get_test_map
from core.feature_extraction import FeatureExtractor
from igp2.maneuver import Maneuver, FollowLane, ManeuverConfig
from test_feature_extraction import get_feature_extractor


ModuleNotFoundError: No module named 'igp2.maneuver'

In [ ]:
lanelet_map = get_test_map()
feature_extractor = FeatureExtractor(lanelet_map)
initial_lanelet = feature_extractor.lanelet_map.laneletLayer.get(2)
final_lanelet = feature_extractor.lanelet_map.laneletLayer.get(5)
route = feature_extractor.routing_graph.getRoute(initial_lanelet, final_lanelet)
path = route.shortestPath()

In [ ]:
final_point = path[-1].centerline[-1]
points = [(p.x, p.y) for l in path for p in list(l.centerline)[:-1]] \
         + [(final_point.x, final_point.y)]

#points = [(0, 1)] + points
x, y = list(zip(*points))

In [ ]:
for l in lanelet_map.laneletLayer:
    LaneletHelpers.plot(l)
    
plt.plot(x, y, '-o')

In [ ]:
t = np.concatenate(([0], np.cumsum(np.linalg.norm(np.diff(points, axis=0), axis=1))))

In [ ]:
plt.plot(t, x, '-o')
plt.plot(t, y, '-o')

In [ ]:
cs_x = CubicSpline(t, x)
cs_y = CubicSpline(t, y)

In [ ]:
ts = np.linspace(t[0], t[-1])
plt.plot(t, x, 'o')
plt.plot(ts, cs_x(ts))

In [ ]:
plt.plot(t, y, 'o')
plt.plot(ts, cs_y(ts))

In [ ]:
for l in lanelet_map.laneletLayer:
    LaneletHelpers.plot(l)

plt.plot(x, y, '-o')
plt.plot(cs_x(ts), cs_y(ts))

In [ ]:
cs_x = CubicSpline(t, x, bc_type=((1, 1), (1, 1)))
cs_y = CubicSpline(t, y, bc_type=((1, 0.5), (1, 1)))

In [ ]:
points_array = np.array(points)

In [ ]:
points_array.shape

In [ ]:
for l in lanelet_map.laneletLayer:
    LaneletHelpers.plot(l)

plt.plot(x, y, '-o')
plt.plot(cs_x(ts), cs_y(ts))

In [ ]:
np.vstack((cs_x(ts), cs_y(ts))).T

In [ ]:
final_direction = np.diff(np.array(points)[-2:], axis=0).flatten()
final_direction = final_direction / np.linalg.norm(final_direction)

In [ ]:
final_direction

In [ ]:
heading = np.pi/20
initial_direction = np.array([np.cos(heading), np.sin(heading)])
initial_direction

In [ ]:
spl_x = make_interp_spline(t, x, bc_type=([(2, 0)], [(2, 0)]))
spl_y = make_interp_spline(t, y, bc_type=([(2, 0)], [(2, 0)]))

for l in lanelet_map.laneletLayer:
    LaneletHelpers.plot(l)

plt.plot(x, y, '-o')
plt.plot(spl_x(ts), spl_y(ts))

In [ ]:
# smoothed spline
s = 1
spl_x = splrep(t, x, s=s)
spl_y = splrep(t, y, s=s)

xs = splev(ts, spl_x)
ys = splev(ts, spl_y)


for l in lanelet_map.laneletLayer:
    LaneletHelpers.plot(l)

plt.plot(x, y, '-o')
plt.plot(xs, ys)

In [ ]:
spl_x[0]

In [ ]:
spl_y[0]

In [ ]:
t

In [ ]:
# b spline
l = len(x) 
t2 = np.linspace(0, 1, l - 2, endpoint=True)
t2=np.append([0,0,0],t2)
t2=np.append(t2,[1,1,1])
tck = [t2, [x, y], 3]

u3 = np.linspace(0,1,(max(l*2,70)),endpoint=True)

xs, ys = splev(u3, tck)


for l in lanelet_map.laneletLayer:
    LaneletHelpers.plot(l)
    
plt.plot(x, y, '-o')
plt.plot(xs, ys)

In [ ]:
x

In [ ]:
final_point = path[-1].centerline[-1]
points = [(p.x, p.y) for l in path for p in list(l.centerline)[:-1]] \
         + [(final_point.x, final_point.y)]
points[0] = (0, 1)
x, y = list(zip(*points))

In [ ]:
cs_x = CubicSpline(t, x, bc_type=((1, 1), (1, 1)))
cs_y = CubicSpline(t, y, bc_type=((1, 0.5), (1, 1)))

In [ ]:
for l in lanelet_map.laneletLayer:
    LaneletHelpers.plot(l)

plt.plot(x, y, '-o')
plt.plot(cs_x(ts), cs_y(ts))

In [ ]:
ls = LineString(points)

In [ ]:
point = Point((4, 5))

In [ ]:
ls.project(point)

In [ ]:
p = ls.interpolate(0.44)

In [ ]:
p.x, p.y

In [ ]:
ls.distance(p)

In [ ]:
result = split(ls, snap(p, ls, 1e-4))

In [ ]:
result.wkt

In [ ]:
snap(p, ls, 1e-4)

In [ ]:
pt = Point((1, 1))
line = LineString([(0,0), (2,2)])
result = split(line, pt)
result.wkt

In [ ]:
# trim ls to length
current_point = Point((1.5, 1.2))
lat_dist = ls.distance(current_point)
long_dist = ls.project(current_point)
margin = 1

for p in ls.coords:
    if ls.project(Point(p)) > lat_dist + long_dist + margin:
        first_point = Point(p)
        break
else:
    print('no points')
    
# TODO check if first point is the final point

In [ ]:
for p in ls.coords:
    print(p)

In [ ]:
ls.coords[-1]

In [ ]:
first_point.coords[0]

In [ ]:
split(ls, first_point).wkt

In [ ]:
feature_extractor = get_feature_extractor()
frame = Frame(0)
state = AgentState(0, 0.1, 1.5, 0, 0, 0, 0, 0, 0, 0, 0, 0)
frame.add_agent_state(0, state)
config = ManeuverConfig({'termination_point': (3.5, 2.9),
                         'initial_lanelet_id': 2,
                         'final_lanelet_id': 5})
maneuver = FollowLane(feature_extractor, config)

path = maneuver.get_path(0, frame, feature_extractor)


In [ ]:
for l in lanelet_map.laneletLayer:
    LaneletHelpers.plot(l)

plt.plot(x, y, '-o')
plt.plot(path[:, 0], path[:, 1])

In [ ]:
a = np.array([1,2,3,4])

In [ ]:
a

In [ ]:
(a[2:] - a[:-2])/2

In [ ]:
a[2:] - 2 * a[1:-1] + a[:-2]

In [ ]:
gamma = np.array(path)
s = np.concatenate(([0], np.cumsum(np.linalg.norm(np.diff(gamma, axis=0), axis=1))))
ds = np.gradient(s).reshape((-1, 1))
d_gamma_ds = np.gradient(gamma, axis=0) / ds
d_2_gamma_ds_2 = np.gradient(d_gamma_ds, axis=0) / ds
kappa = np.linalg.det(np.dstack([d_gamma_ds, d_2_gamma_ds_2])) \
    / np.linalg.norm(d_gamma_ds, axis=1) ** 3

In [ ]:
ds.reshape((1, -1))

In [ ]:
kappa

In [ ]:
plt.plot(kappa)

In [ ]:
plt.plot(path[:,0], path[:,1])

In [ ]:
t

In [ ]:
np.gradient(t)

In [ ]:
feature_extractor = get_feature_extractor()
frame = Frame(0)
state = AgentState(0, 0.1, 1.5, 0, 0, 0, 0, 0, 0, 0, 0, 0)
frame.add_agent_state(0, state)
config = ManeuverConfig({'termination_point': (3.5, 2.9),
                         'initial_lanelet_id': 2,
                         'final_lanelet_id': 5})
maneuver = FollowLane(feature_extractor, config)

path, velocity = maneuver.get_trajectory(0, frame, feature_extractor)
for l in lanelet_map.laneletLayer:
    LaneletHelpers.plot(l)

plt.plot(x, y, '-o')
plt.plot(path[:, 0], path[:, 1])

In [ ]:
velocity

In [ ]:
scenario = Scenario.load('../scenario_config/heckstrasse.json')

In [ ]:
plt.figure(figsize=(12, 8))
scenario.plot()

In [ ]:
feature_extractor = FeatureExtractor(scenario.lanelet_map)

In [6]:
from core.data_processing import get_goal_priors, get_dataset
for scenario in ['heckstrasse', 'bendplatz', 'frankenberg']:
    data_subsets = ['train', 'valid', 'test']
    dataset = pd.concat([get_dataset(scenario, subset) for subset in data_subsets])
    agent_counts = dataset[['agent_id', 'episode'
                           ]].drop_duplicates().episode.value_counts()
    print(scenario)
    print(agent_counts.sum())
    print(agent_counts)
    

heckstrasse
884
2    358
0    345
1    181
Name: episode, dtype: int64
bendplatz
2014
1     254
5     219
10    203
6     201
7     185
4     181
9     172
8     166
3     157
2     149
0     127
Name: episode, dtype: int64
frankenberg
1556
4     185
10    179
7     168
5     166
9     162
8     153
3     128
2     121
0     119
6      98
1      77
Name: episode, dtype: int64


In [2]:
from core.data_processing import get_goal_priors, get_dataset
for scenario in ['heckstrasse', 'bendplatz', 'frankenberg']:
    data_subsets = ['train', 'valid', 'test']
    dataset = pd.concat([get_dataset(scenario, subset) for subset in data_subsets])
    agent_counts = dataset[['agent_id', 'episode'
                           ]].drop_duplicates().episode.value_counts()
    print(scenario)
    print(agent_counts.sum())
    print(agent_counts)
    

heckstrasse
901
2    361
0    351
1    189
Name: episode, dtype: int64


FileNotFoundError: File b'/home/s1686074/projects/av-goal-recognition/data/bendplatz_e7.csv' does not exist